In [185]:
# from pm4py.objects.log.importer.xes import importer as xes_importer
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import datetime
import random
import time
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn import preprocessing as pre
from sklearn import tree
import matplotlib.pyplot as plt
from collections import Counter
import math
import tracemalloc
import psutil

In [186]:
def fix_time(time):
    return (datetime.datetime.fromisoformat(time))

def load_data(BPI = 'BPI.csv', BPI_attr = 'BPI_attr.csv',  data2012 = False, sample=False):
    df_BPI = pd.read_csv(BPI)
    df_BPI_attr = pd.read_csv(BPI_attr)
    
    if 'Unnamed: 0' in df_BPI.columns:
        df_BPI = df_BPI.rename(columns={'Unnamed: 0': 'case_id', 'Unnamed: 1': 'step_number'})
        df_BPI_attr = df_BPI_attr.rename(columns={'Unnamed: 0': 'case_id'})
    
    df_BPI['time:timestamp'] = df_BPI['time:timestamp'].apply(fix_time)

    df_BPI_attr['REG_DATE'] = df_BPI_attr['REG_DATE'].apply(fix_time)
    
    df_BPI['time:weekday'] = [x.weekday() for x in df_BPI['time:timestamp']]
    df_BPI['time:hour'] = [x.hour for x in df_BPI['time:timestamp']]
    df_BPI['time:day'] = [str(x.day) + '-' + str(x.month) + '-' + str(x.year) for x in df_BPI['time:timestamp']]
    df_BPI['time:event_count'] = df_BPI.groupby('time:day')['time:day'].transform('count')
    df_BPI['time:busy_day'] = df_BPI['time:event_count'] > 2525
    
    if(sample):
        df_BPI, df_BPI_attr = df_BPI[:50000], df_BPI_attr[:2359]
    
    return (df_BPI, df_BPI_attr)

def load_data_xes(data):
    BPI = xes_importer.apply(data)

In [187]:
df, df_attr = load_data(BPI = 'Datasets/BPI_2012.csv', BPI_attr = 'Datasets/BPI_attr_2012.csv', sample=False)

In [188]:
df_train, df_validation, df_test = pd.read_csv('Datasets/trainExample.csv'), pd.read_csv('Datasets/validationExample.csv'), pd.read_csv('Datasets/testExample.csv')

In [189]:
df['combined_names'] = df['lifecycle:transition'] + ' + ' + df['concept:name']
    
#Prepare the time between columns
if (type(df['time:timestamp'].iloc[0]) != datetime.datetime):
    df['time:timestamp'] = df['time:timestamp'].apply(fix_time)

df["time:time_for_next"] = df["time:timestamp"].diff()
df.loc[df['step_number'] == 0, 'time:time_for_next'] = pd.Timedelta(0) #Changed it a bit to always insert 0 into the first row
df["time:time_for_next"] = [int(x.total_seconds()) for x in df["time:time_for_next"]]
df["time:time_for_next"] = df["time:time_for_next"].shift(-1)

#Prepare the next event colum
df["next_event"] = df["combined_names"]
df.loc[df['step_number'] == 0, 'next_event'] = 'editor: close_case'
df["next_event"] = df["next_event"].shift(-1)
df.loc[len(df) - 1, 'next_event'] = 'editor: close_case'

In [190]:
df

,case_id,step_number,org:resource,lifecycle:transition,concept:name,time:timestamp,time:weekday,time:hour,time:day,time:event_count,time:busy_day,combined_names,time:time_for_next,next_event
0,0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,5,0,1-10-2011,381,False,COMPLETE + A_SUBMITTED,0.0,COMPLETE + A_PARTLYSUBMITTED
1,0,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,5,0,1-10-2011,381,False,COMPLETE + A_PARTLYSUBMITTED,53.0,COMPLETE + A_PREACCEPTED
2,0,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,5,0,1-10-2011,381,False,COMPLETE + A_PREACCEPTED,0.0,SCHEDULE + W_Completeren aanvraag
3,0,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,5,0,1-10-2011,381,False,SCHEDULE + W_Completeren aanvraag,39427.0,START + W_Completeren aanvraag
4,0,4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,5,11,1-10-2011,381,False,START + W_Completeren aanvraag,356.0,COMPLETE + A_ACCEPTED
5,0,5,10862.0,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00,5,11,1-10-2011,381,False,COMPLETE + A_ACCEPTED,145.0,COMPLETE + O_SELECTED
6,0,6,10862.0,COMPLETE,O_SELECTED,2011-10-01 11:45:09.243000+02:00,5,11,1-10-2011,381,False,COMPLETE + O_SELECTED,0.0,COMPLETE + A_FINALIZED
7,0,7,10862.0,COMPLETE,A_FINALIZED,2011-10-01 11:45:09.243000+02:00,5,11,1-10-2011,381,False,COMPLETE + A_FINALIZED,1.0,COMPLETE + O_CREATED
8,0,8,10862.0,COMPLETE,O_CREATED,2011-10-01 11:45:11.197000+02:00,5,11,1-10-2011,381,False,COMPLETE + O_CREATED,0.0,COMPLETE + O_SENT
9,0,9,10862.0,COMPLETE,O_SENT,2011-10-01 11:45:11.380000+02:00,5,11,1-10-2011,381,False,COMPLETE + O_SENT,0.0,SCHEDULE + W_Nabellen offertes


In [191]:
def timeChain(df):
    
    df_chain = df.copy()
    shape = df_chain.shape[0]
    
    chain = {}
    concept_names = ['editor:first_event'] + list(df_chain['concept:name'])
    times_list = ['editor:first_time'] + list(df_chain['time:time_for_next'])
    id_list = ['editor:first_id'] + list(df_chain['case_id']) #Otherwise we check i+1-th position that does not exist

    for i in range(len(concept_names)):

        if shape > i + 2:
            key1 = concept_names[i]
            key2 = concept_names[i+1]
            event = concept_names[i+2]
            time1 = times_list[i]
            time2 = times_list[i+1]
            
        

            if (id_list[i] == id_list[i+2]): 

                if ((key1, key2) not in chain):
                    chain[(key1, key2)] = [time2] 
                else:
                    chain[(key1, key2)].append(time2)

            elif (event == 'A_SUBMITTED'): 

                if ((key1, key2) not in chain):
                    chain[(key1, key2)] = [0]
                else:
                    chain[(key1, key2)].append(0)

            elif (key2 == 'A_SUBMITTED'):

                if ((key1, key2) not in chain):
                    chain[(key1, key2)] = [time1]
                else:
                    chain[(key1, key2)].append(time1)
    
    print('Chain size: {0} distinct event pairs.'.format(len(chain)))
    
    return chain

def getPredictions(df, chain):
    df_2markov = df.copy()
    nextEvents = []

    for i in tqdm(range(-1, df_2markov.shape[0] - 1)):

        if (i == -1):
            nextEvents.append(0)
        else:
            event1 = df_2markov['concept:name'][i]
            event2 = df_2markov['concept:name'][i+1]
        
            if ((event1, event2) in chain):
                nextEvents.append(random.choice(chain[(event1, event2)]))
            else:
                nextEvents.append(event2) #only for the code to work
        
        
        
    return nextEvents

In [192]:
chain = timeChain(df)
chain

Chain size: 137 distinct event pairs.


{('editor:first_event', 'A_SUBMITTED'): ['editor:first_time'],
 ('A_SUBMITTED', 'A_PARTLYSUBMITTED'): [53.0,
  54.0,
  42.0,
  37.0,
  40.0,
  52.0,
  41.0,
  43.0,
  13.0,
  31.0,
  41.0,
  42.0,
  46.0,
  43.0,
  50.0,
  40.0,
  45.0,
  43.0,
  15.0,
  13.0,
  46.0,
  41.0,
  40.0,
  37.0,
  39.0,
  11.0,
  42.0,
  36.0,
  35.0,
  31.0,
  39.0,
  39.0,
  45.0,
  44.0,
  42.0,
  42.0,
  41.0,
  50.0,
  46.0,
  49.0,
  40.0,
  36.0,
  10.0,
  46.0,
  43.0,
  37.0,
  16.0,
  54.0,
  44.0,
  49.0,
  41.0,
  32.0,
  48.0,
  44.0,
  39.0,
  33.0,
  14.0,
  14.0,
  40.0,
  45.0,
  44.0,
  41.0,
  2.0,
  41.0,
  8.0,
  2.0,
  38.0,
  15.0,
  36.0,
  40.0,
  12.0,
  48.0,
  9.0,
  40.0,
  42.0,
  35.0,
  11.0,
  34.0,
  40.0,
  10.0,
  54.0,
  37.0,
  13.0,
  42.0,
  40.0,
  43.0,
  39.0,
  15.0,
  50.0,
  42.0,
  33.0,
  10.0,
  45.0,
  42.0,
  50.0,
  43.0,
  34.0,
  37.0,
  51.0,
  43.0,
  35.0,
  43.0,
  16.0,
  43.0,
  47.0,
  10.0,
  38.0,
  9.0,
  45.0,
  21.0,
  44.0,
  44.0,
  42.0,


In [193]:
eventPredictions = getPredictions(df, chain)

df['time:prediction_for_next'] = eventPredictions

100%|███████████████████████████████████████████████████████████████████████| 262200/262200 [00:05<00:00, 48908.03it/s]


In [194]:
df

,case_id,step_number,org:resource,lifecycle:transition,concept:name,time:timestamp,time:weekday,time:hour,time:day,time:event_count,time:busy_day,combined_names,time:time_for_next,next_event,time:prediction_for_next
0,0,0,112.0,COMPLETE,A_SUBMITTED,2011-10-01 00:38:44.546000+02:00,5,0,1-10-2011,381,False,COMPLETE + A_SUBMITTED,0.0,COMPLETE + A_PARTLYSUBMITTED,0.0
1,0,1,112.0,COMPLETE,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880000+02:00,5,0,1-10-2011,381,False,COMPLETE + A_PARTLYSUBMITTED,53.0,COMPLETE + A_PREACCEPTED,9.0
2,0,2,112.0,COMPLETE,A_PREACCEPTED,2011-10-01 00:39:37.906000+02:00,5,0,1-10-2011,381,False,COMPLETE + A_PREACCEPTED,0.0,SCHEDULE + W_Completeren aanvraag,0.0
3,0,3,112.0,SCHEDULE,W_Completeren aanvraag,2011-10-01 00:39:38.875000+02:00,5,0,1-10-2011,381,False,SCHEDULE + W_Completeren aanvraag,39427.0,START + W_Completeren aanvraag,29.0
4,0,4,NaN,START,W_Completeren aanvraag,2011-10-01 11:36:46.437000+02:00,5,11,1-10-2011,381,False,START + W_Completeren aanvraag,356.0,COMPLETE + A_ACCEPTED,869.0
5,0,5,10862.0,COMPLETE,A_ACCEPTED,2011-10-01 11:42:43.308000+02:00,5,11,1-10-2011,381,False,COMPLETE + A_ACCEPTED,145.0,COMPLETE + O_SELECTED,171.0
6,0,6,10862.0,COMPLETE,O_SELECTED,2011-10-01 11:45:09.243000+02:00,5,11,1-10-2011,381,False,COMPLETE + O_SELECTED,0.0,COMPLETE + A_FINALIZED,0.0
7,0,7,10862.0,COMPLETE,A_FINALIZED,2011-10-01 11:45:09.243000+02:00,5,11,1-10-2011,381,False,COMPLETE + A_FINALIZED,1.0,COMPLETE + O_CREATED,1.0
8,0,8,10862.0,COMPLETE,O_CREATED,2011-10-01 11:45:11.197000+02:00,5,11,1-10-2011,381,False,COMPLETE + O_CREATED,0.0,COMPLETE + O_SENT,0.0
9,0,9,10862.0,COMPLETE,O_SENT,2011-10-01 11:45:11.380000+02:00,5,11,1-10-2011,381,False,COMPLETE + O_SENT,0.0,SCHEDULE + W_Nabellen offertes,0.0


In [195]:
df_testing = df.copy()
df_testing['time:diff'] = abs(df_testing['time:time_for_next'] - df_testing['time:prediction_for_next'])
df_testing['time:diff'].median()

82.0

In [198]:
df_testing['time:diff'].mean()

56657.20975289761

In [197]:
df_testing['time:diff'].median()

82.0